In [1]:
from ipywidgets import FileUpload
import os
import re

import numpy as np
import pandas as pd
from pandas import DataFrame

from scipy import ndimage as ndi
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from scipy.signal import chirp, find_peaks, peak_widths, savgol_filter
from skimage.filters import sobel, gaussian
from skimage import measure

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly import tools
init_notebook_mode(connected=True)


import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
from collections import OrderedDict
from numpy import *
from IPython.display import FileLink, FileLinks, clear_output
from string import Template
from hublib.ui import FileUpload, Download

import os,sys
sys.path.insert(0, os.path.abspath('..'))
from ipywidgets import HBox

<IPython.core.display.Javascript object>

In [2]:
# called when all files finish uploading
def done_cb(w, name):
    # Do something with the files here...
    # We just print their names
    print("%s uploaded" % name)
    a=name
    # reset clears and re-enables the widget for more uploads
    # You may want to do this somewhere else, depending on your GUI
    w.reset()

In [3]:
def location_sort(loc_list, ncol = 5, nrow = 5):
    loc_list = np.array(loc_list)
    sort_index = np.argsort(loc_list[0:ncol,1])[::-1]
    for i in range(1,nrow):
        sort_index = np.append(sort_index,ncol*i+np.argsort(loc_list[(ncol*i):(ncol*i+ncol),1])[::-1])
    return(sort_index)

In [4]:
# input section
text_layout1 = widgets.Layout(width ='145px', position = 'middle')
Sample_number= widgets.BoundedFloatText(
    value=1,
    min=1,
    max=9,
    step=1,
    layout=text_layout1,
    tooltip = 'Sample File Number'
)

vinputa3 = widgets.VBox([Sample_number]) 
#Designed_Radius_label = widgets.Label("Designed Radius (mm):")
dimlabel3 = widgets.HTML('<p style="background:lightgray" align="center">Download Sample </p>')

#vlabelsa1 = widgets.VBox([Designed_Radius_label])
inputs2a3 = widgets.HBox([vinputa3],layout=widgets.Layout(border='none'))


inputs23 = widgets.VBox([inputs2a3])
input3 = widgets.VBox([dimlabel3,inputs23], layout={'width': '150px', 'position': 'middle'})




In [5]:
text_layout1 = widgets.Layout(width ='145px', position = 'middle')
Designed_Radius= widgets.BoundedFloatText(
    value=2.,
    min=0.1,
    max=3.2,
    step=0.1,
    layout=text_layout1,
    tooltip = 'Designed_Radius'
)

vinputa1 = widgets.VBox([Designed_Radius]) 
#Designed_Radius_label = widgets.Label("Designed Radius (mm):")
dimlabel1 = widgets.HTML('<p style="background:lightgray" align="center"> Radius in Design (&mu;m)</p>')

#vlabelsa1 = widgets.VBox([Designed_Radius_label])
inputs2a1 = widgets.HBox([vinputa1],layout=widgets.Layout(border='none'))


inputs21 = widgets.VBox([inputs2a1])
input2 = widgets.VBox([dimlabel1,inputs21], layout={'width': '150px', 'position': 'middle'})



#input section
text_layout1 = widgets.Layout(width ='45px',position = 'middle')
column_index= widgets.BoundedFloatText(
    value=1.,
    min=1.,
    max=5.,
    step=1.,
    layout=text_layout1,
    tooltip = 'Column Number'
)

# Laser exposure time
row_index = widgets.BoundedFloatText(
    value = 1.,
    min = 1.,
    max = 5.,
    step = 1.,
    layout=text_layout1,
    tooltip = 'Row number'
)


column_label = widgets.Label("X:")
row_label=widgets.Label(" Y:")
dimlabel = widgets.HTML('<p style="background:lightgray" align="center"> Structure Index</p>')

row_input = widgets.HBox([row_label, row_index]) 
col_input = widgets.HBox([column_label,column_index])
inputs2a = widgets.HBox([col_input, row_input],layout=widgets.Layout(border='none'))


inputs2 = widgets.VBox([inputs2a])
input1 = widgets.VBox([dimlabel,inputs2], layout={'width': '150px'})



In [6]:
def area_segment(filename):
    img_raw = np.loadtxt(filename, delimiter = ',')
    img_array = img_raw.reshape([-1])
    img_gaussian = ndi.filters.gaussian_filter(img_raw,2, mode='nearest') 
    img_array = img_gaussian.reshape([-1])

    num_bins = int((img_array.max()-img_array.min())*1000)
    hist, bin_edges = np.histogram(img_array, bins=num_bins)

    hist_filtered = ndi.filters.gaussian_filter1d(hist,15)
    hist_filtered = hist_filtered
    peaks, _ = find_peaks(hist_filtered, height = 100, distance = 50)
    results_half = peak_widths(hist_filtered, peaks, rel_height=0.5)
    results_full = peak_widths(hist_filtered, peaks, rel_height=0.95)

    th1 = bin_edges[int(results_full[3][0])]
    th2 = bin_edges[int(results_full[2][1])]
    th3 = bin_edges[int(results_full[3][1])]

    markers = img_gaussian > th3 + 0.5 #2.8 #(th3*1+th2)/2
    background = img_gaussian > (th2+th1)/2
    final_mask = False & markers

    label_objects, nb_labels = ndi.label(background)

    sizes = np.bincount(label_objects.ravel())
    mask_sizes = sizes > 300
    mask_sizes[0] = 0
    mask_cleaned = mask_sizes[label_objects]

    label_objects, nb_labels = ndi.label(markers)

    sizes = np.bincount(label_objects.ravel())
    mask_sizes = sizes > 300
    mask_sizes[0] = 0
    mask_cleaned = mask_sizes[label_objects]

    label_objects, nb_labels = ndi.label(mask_cleaned)
    center_temp = ndi.center_of_mass(mask_cleaned, label_objects, np.arange(1,26))
    center_temp = np.array(center_temp)
    center_temp_index = location_sort(center_temp)
    center_temp = center_temp[center_temp_index]
    
    num_pixel = np.zeros([25]) 
    surround_height = np.zeros([25]) 
    surround_height_std = np.zeros([25]) 

    center_index = np.argsort(np.append([0],center_temp_index+1))

    label_objects = center_index[label_objects]
    
    for sphere_index in range(25):
        center_y, center_x = center_temp[sphere_index]
        center_y = int(center_y)
        center_x = int(center_x)

        roi_current = img_gaussian[(center_y-40):(center_y+40),(center_x-40):(center_x+40)]
        raw_current = img_raw[(center_y-40):(center_y+40),(center_x-40):(center_x+40)]

        sobel_current = sobel(roi_current)
        sobel_array = sobel_current.reshape(-1)

        struc_region = sobel_current>0.1
        struc_region = ndi.binary_fill_holes(struc_region)
        
        #struc_region = ndi.binary_opening(struc_region,iterations=4)
        #struc_region = ndi.binary_closing(struc_region,iterations=5)
        #struc_region = ndi.binary_erosion(struc_region,iterations=5)
        #struc_region = ndi.binary_dilation(struc_region,iterations=1)

        roi_plot1 = raw_current + 0
        roi_plot1[struc_region] = 0.3
        roi_plot2 = raw_current + 0
        roi_plot2[True ^ struc_region] = 0.0

        final_mask[(center_y-40):(center_y+40),(center_x-40):(center_x+40)] = struc_region

        num_pixel[sphere_index] = struc_region.sum()
        surround_height[sphere_index] = raw_current[True ^ struc_region].mean()
        surround_height_std[sphere_index] = raw_current[True ^ struc_region].std()
    #print(csvlist[current_img_index])
    
    label_objects, nb_labels = ndi.label(final_mask)

    sizes = np.bincount(label_objects.ravel())
    mask_sizes = (sizes > 900) 
    mask_sizes[0] = 0
    mask_cleaned = mask_sizes[label_objects]

    label_objects, nb_labels = ndi.label(mask_cleaned)
    center = ndi.center_of_mass(mask_cleaned, label_objects, np.arange(1,26))
    center_index = np.argsort(np.append([0],location_sort(center)+1))
    
    label_objects = center_index[label_objects]

    return label_objects

In [7]:
def measurement(filename, label_objects):
    img_raw = np.loadtxt(filename, delimiter = ',')

    hemisphere_template = np.zeros([30,30])
    XX = np.arange(-30, 30)*0.0948
    YY = np.arange(-30, 30)*0.0948
    XX, YY = np.meshgrid(XX, YY)
    dist = XX*XX+YY*YY
    dist[dist > Designed_Radius.value**2] = Designed_Radius.value**2
    hemisphere_template = np.sqrt(Designed_Radius.value**2-dist)
    scan_rate, power = 30, 40
    
    design, run_order = [int(s) for s in re.findall("\d+", filename)]
    objectprops = measure.regionprops(label_objects, img_raw)

    label = np.zeros([25]) 
    
    centroid_x = np.zeros([25]) 
    centroid_y = np.zeros([25]) 
    
    num_pixel = np.zeros([25]) 
    area = np.zeros([25]) 
    equivalent_diameter = np.zeros([25]) 
    
    major_axis_length = np.zeros([25]) 
    minor_axis_length = np.zeros([25]) 
    orientation = np.zeros([25]) 
    eccentricity = np.zeros([25]) 

    surround_height_avg = np.zeros([25]) 
    surround_height_std = np.zeros([25]) 
    height_max_raw = np.zeros([25]) 
    height_avg_raw = np.zeros([25]) 
    height_max_rela = np.zeros([25]) 
    height_avg_rela = np.zeros([25]) 
    volume_raw = np.zeros([25])
    volume_rela = np.zeros([25])
    volume_redundant = np.zeros([25])
    volume_redundant = np.zeros([25])
    redundant_volume = np.zeros([25])
    unformed_volume = np.zeros([25])
    absolute_error = np.zeros([25])

    for sphere_index in range(25):
        label[sphere_index] = objectprops[sphere_index].label
        center_y, center_x = objectprops[sphere_index].weighted_centroid
        centroid_x[sphere_index] = center_x
        centroid_y[sphere_index] = center_y
        center_y = int(center_y)
        center_x = int(center_x)

        raw_current = img_raw[(center_y-30):(center_y+30),(center_x-30):(center_x+30)]
        struc_region = (label_objects[(center_y-30):(center_y+30),(center_x-30):(center_x+30)] > 0)

        num_pixel[sphere_index] = objectprops[sphere_index].area
        area[sphere_index] = num_pixel[sphere_index]*0.0948*0.0948
        equivalent_diameter[sphere_index] = objectprops[sphere_index].equivalent_diameter*0.0948

        major_axis_length[sphere_index] = objectprops[sphere_index].major_axis_length*0.0948
        minor_axis_length[sphere_index] = objectprops[sphere_index].minor_axis_length*0.0948
        orientation[sphere_index] = objectprops[sphere_index].orientation
        eccentricity[sphere_index] = objectprops[sphere_index].eccentricity

        surround_height = raw_current[True ^ struc_region]
        #surround_height = surround_height[surround_height]
        
        #if np.abs(surround_height.mean() - np.median(surround_height)) > 0.019:
        #    print(surround_height.mean() - np.median(surround_height))
        
        surround_height_avg[sphere_index] = surround_height.mean() 
        surround_height_std[sphere_index] = surround_height.std()
        height_max_raw[sphere_index] = objectprops[sphere_index].max_intensity
        height_avg_raw[sphere_index] = objectprops[sphere_index].mean_intensity
        volume_raw[sphere_index] = height_avg_raw[sphere_index]*area[sphere_index]

        height_max_rela[sphere_index] = objectprops[sphere_index].max_intensity - surround_height_avg[sphere_index]
        height_avg_rela[sphere_index] = objectprops[sphere_index].mean_intensity - surround_height_avg[sphere_index]
        volume_rela[sphere_index] = height_avg_rela[sphere_index]*area[sphere_index]
        
        error = raw_current-(hemisphere_template+surround_height_avg[sphere_index])
        error = error[struc_region]
        redundant_volume[sphere_index] = error[error>0].sum()*0.0948*0.0948
        unformed_volume[sphere_index] = error[error<0].sum()*0.0948*0.0948
        absolute_error[sphere_index] = redundant_volume[sphere_index] - unformed_volume[sphere_index]
        
    scan_rate_array = np.repeat(scan_rate, 25)
    power_array = np.repeat(power, 25)
    run_order_array = np.repeat(run_order, 25)
    design_array = np.repeat(design, 25)
    
    x,y = np.meshgrid(np.arange(1,6),np.arange(1,6))
    dist = np.reshape(np.sqrt((x-3)*(x-3)+(y-3)*(y-3)),-1)
    
    x = (-label) % 5 + 1
    y = 5 - (label - 1) // 5
    
    data = {'scan_rate': scan_rate_array,
            'design': design_array,
            'power': power_array,
            'centroid_x': centroid_x,
            'centroid_y': centroid_y,
            'run_order': run_order_array,
            'index': label,
            'index_x': np.reshape(x,-1),
            'index_y': np.reshape(y,-1),
            'dist2center': dist,
            'num_pixel': num_pixel,
            'area': area,
            'equivalent_diameter': equivalent_diameter,
            'major_axis_length': major_axis_length,
            'minor_axis_length': minor_axis_length,
            'orientation': orientation,
            'eccentricity': eccentricity,
            'surround_height_avg': surround_height_avg,
            'surround_height_std': surround_height_std,
            'height_max_raw': height_max_raw,
            'height_avg_raw': height_avg_raw,
            'volume_raw': volume_raw,
            'height_max_rela': height_max_rela,
            'height_avg_rela': height_avg_rela,
            'volume_rela': volume_rela,
            'redundant_volume': redundant_volume,
            'unformed_volume': unformed_volume,
            'absolute_error': absolute_error}
    df = DataFrame(data)
    
    feature_file_path = 'download/' + filename[11:]
    df.to_csv(feature_file_path, index = False)

    return feature_file_path

In [8]:
def plot_trend(feature_file_path):
    feature_df = pd.read_csv(feature_file_path)
    trace1 = go.Scatter(
        x = (-feature_df['index']) % 5 + 1,
        y = 5-(feature_df['index']-1)//5 ,
        mode='markers',
        marker=dict(
            size = (feature_df['height_max_rela']-1.55)*250,
            color = feature_df['height_max_rela'], 
            colorscale='Viridis',
            showscale=True
        ),
        colorbar_x=0.45
    )
    data = [trace1]

    layout1 = go.Layout(
        title = 'Height',
        xaxis=dict(
            title='X axis',
            tick0 = 1,
            dtick = 1
        ),
        yaxis=dict(
            title='Y axis'
        ),
        width=450, height=450,
        font=dict(
            size=18,
        ),
    )

    #fig1 = go.Figure(data=data, layout=layout)
    #iplot(fig1)
    
    #new plot
    trace2 = go.Scatter(
        x = (-feature_df['index']) % 5 + 1,
        y = 5-(feature_df['index']-1)//5 ,
        mode='markers',
        marker=dict(
            size = (Designed_Radius.value-1.8-(feature_df['height_max_rela']-1.55))*350,
            color = Designed_Radius.value-feature_df['height_max_rela'], 
            colorscale='Viridis',
            showscale=True
        )
    )

    layout2 = go.Layout(
        title = 'Height Error',
        xaxis=dict(
            title='X axis',
            tick0 = 1,
            dtick = 1
        ),
        yaxis=dict(
            title='Y axis'
        ),
        width=450, height=450,
        font=dict(
            size=18,
        ),
    )
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)

    #fig2 = go.Figure(data=data, layout=layout)
    
    iplot(fig)
    

In [9]:
def measurement_analystic(out):
    filename =f.list()[0]
    label_objects = area_segment(filename)  
        
    #plt.imshow(label_objects)
    #plt.show()
    
    feature_file_path = measurement(filename, label_objects)
    plot_trend(feature_file_path)
    
    filename ='download/'+f.list()[0][11:]
    Download(filename, style='info')

In [10]:
f = FileUpload("TPL Measurement Data:", 
               "This is a description that appears when the mouse hovers over the name.",
               dir='uploadpath',
               cb=done_cb,
               maxsize='10M')

In [11]:
#Output section
text_layout2 = widgets.Layout(width ='80px', position = 'left')

op1label = widgets.Label("Height: ")
op2label = widgets.Label("Radius: ")
op3label = widgets.Label("Volume: ")

op4label = widgets.Label("Height Error: ")
op5label = widgets.Label("Radius Error: ")
op6label = widgets.Label("Volume Error: ")

op7label = widgets.Label("Height Error Percentage(%): ")
op8label = widgets.Label("Radius Error Percentage(%): ")
op9label = widgets.Label("Volume Error Percentage(%): ")

op1 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)
op4 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)
op7 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)

op2 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)
op5 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)

op8 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)

op3 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)
op6 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)
op9 = widgets.Text(
    value = '%.2f'%(0.),
    layout = text_layout2,
    position = 'right'
)

vlabels2_left = widgets.VBox([op1label,op2label,op3label])
vlabels2_center = widgets.VBox([op4label,op5label,op6label])
vlabels2_right = widgets.VBox([op7label,op8label,op9label])
outlabels_left = widgets.VBox([op1,op2,op3])
outlabels_center = widgets.VBox([op4,op5,op6])
outlabels_right = widgets.VBox([op7,op8,op9])
outlabels2 = widgets.HBox([vlabels2_left,outlabels_left,vlabels2_center,outlabels_center,vlabels2_right,outlabels_right])


In [12]:
def new_visualization(out):
    
    filename = f.list()[0]
    feature_file_path = 'download/' + filename[11:]
    feature_df = pd.read_csv(feature_file_path)
    struc_index= (6 - row_index.value)*5 - column_index.value
    
    height, diameter, volume = feature_df.iloc[int(struc_index)][['height_max_rela', 'equivalent_diameter', 'volume_rela']]
    radius = diameter / 2
    
    op1.value = "%0.03f" % height
    op2.value = "%0.03f" % radius
    op3.value = "%0.03f" % volume
#     print("Height: %.3f" % height)
#     print("Radius: %.3f" % radius)
#     print("Volume: %.3f" % volume)
    op4.value = "%0.03f" % (Designed_Radius.value-height)
    op5.value = "%0.03f" % (Designed_Radius.value-radius)
    op6.value = "%0.03f" % (2./3*math.pi*Designed_Radius.value**3-volume)
    
    op7.value = "%0.03f" % (((Designed_Radius.value-height)/height)*100)
    op8.value = "%0.03f" % (((Designed_Radius.value-radius)/radius)*100)
    op9.value = "%0.03f" % (((2./3*math.pi*Designed_Radius.value**3-volume)/volume)*100)
    
    hemisphere_template = np.zeros([35,35])
    XX = np.arange(-35, 35)*0.0948
    YY = np.arange(-35, 35)*0.0948
    XX, YY = np.meshgrid(XX, YY)
    dist = XX*XX+YY*YY
    dist[dist > Designed_Radius.value**2] = Designed_Radius.value**2
    hemisphere_template = np.sqrt(Designed_Radius.value**2-dist)
    
    img_raw = np.loadtxt(filename, delimiter = ',')
    label_objects = area_segment(filename)  
    objectprops = measure.regionprops(label_objects, img_raw)

    center_y, center_x = objectprops[int(struc_index)-1].weighted_centroid
    center_y = int(center_y)
    center_x = int(center_x)
    raw_current = img_raw[(center_y-35):(center_y+35),(center_x-35):(center_x+35)]

    struc_region = (label_objects[(center_y-35):(center_y+35),(center_x-35):(center_x+35)] > 0)
    surround_height_avg = raw_current[True ^ struc_region].mean()

    ZZ = raw_current - surround_height_avg
    ZZ[True ^ struc_region] = 0.0
    
    trace1 = go.Surface(x = XX, y = YY, z=ZZ, colorscale='Viridis', showscale=True)
    trace2 = go.Surface(x = XX, y = YY, z=hemisphere_template, showscale=False, opacity=0.5)

    data = [trace1, trace2]

    layout = go.Layout(
        #title = 'Volume Error',
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        scene = dict(
            xaxis=dict(
                title='X (' + u"\u03BC" + 'm)'
            ),
            yaxis=dict(
                title='Y (' + u"\u03BC" + 'm)'),
            zaxis = dict(
                title='Z (' + u"\u03BC" + 'm)'),
            #aspectratio=dict(x=1, y=1, z=0.5),
            aspectmode='data',
            camera = dict(
                eye=dict(x=2, y=-2, z=1)),
        ),

    )

    fig = dict(data = data, layout = layout)

    iplot(fig)



In [13]:
def plot_trend(feature_file_path):
    feature_df = pd.read_csv(feature_file_path)
    
    trace1 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                feature_df['height_max_rela'].values.reshape(-1,1)
            ).reshape(-1),
            color = feature_df['height_max_rela'], 
            colorscale='Viridis',
            colorbar=dict(
                x = 0.4,
                y = 0.89,
                lenmode='fraction', 
                len=0.23,
            ),
            showscale=True
        ),
    )
    trace2 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                Designed_Radius.value - feature_df['height_max_rela'].values.reshape(-1,1)
            ).reshape(-1),
            color = Designed_Radius.value-feature_df['height_max_rela'], 
            colorscale='Viridis',
            colorbar=dict(
                x = 1,
                y = 0.89,
                lenmode='fraction', 
                len=0.23,
            ),
            showscale=True
        ),
    )
    
    trace3 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                feature_df['equivalent_diameter'].values.reshape(-1,1)
            ).reshape(-1),
            color = feature_df['equivalent_diameter']/2, 
            colorscale='Viridis',
            colorbar=dict(
                x = 0.4,
                y = 0.5,
                lenmode='fraction', 
                len=0.23,
            ),
            showscale=True
        )
    )
    trace4 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                Designed_Radius.value - feature_df['equivalent_diameter'].values.reshape(-1,1)
            ).reshape(-1),
            color = Designed_Radius.value - feature_df['equivalent_diameter']/2, 
            colorscale='Viridis',
            colorbar=dict(
                x = 1,
                y = 0.5,
                lenmode='fraction', 
                len=0.23,
            ),
            showscale=True
        )
    )
    
    trace5 = go.Scatter(
        x = feature_df['index_x'],
        y = feature_df['index_y'],
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                feature_df['volume_rela'].values.reshape(-1,1)
            ).reshape(-1),
            color = feature_df['volume_rela'], 
            colorscale='Viridis',
            colorbar=dict(
                x = 0.4,
                y = 0.11,
                lenmode='fraction', 
                len=0.23,
            ),
            showscale=True
        )
    )
    trace6 = go.Scatter(
        x = (-feature_df['index']) % 5 + 1,
        y = 5-(feature_df['index']-1)//5 ,
        mode='markers',
        marker=dict(
            size = MinMaxScaler((15, 30)).fit_transform(
                (2./3*np.pi*Designed_Radius.value**3)-feature_df['volume_rela'].values.reshape(-1,1)
            ).reshape(-1),
            color = (2./3*np.pi*Designed_Radius.value**3)-feature_df['volume_rela'], 
            colorscale='Viridis',
            colorbar=dict(
                x = 1,
                y = 0.11,
                lenmode='fraction', 
                len=0.23,
            ),
            showscale=True
        )
    )
    
    
    fig = make_subplots(rows=3, cols=2, subplot_titles = [
        'Height', 'Height Error',
        'Radius', 'Radius Error',
        'Volume', 'Volume Error',
    ])
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)
    fig.add_trace(trace3, row=2, col=1)
    fig.add_trace(trace4, row=2, col=2)
    fig.add_trace(trace5, row=3, col=1)
    fig.add_trace(trace6, row=3, col=2)
    
    fig.update_xaxes(title_text="X axis", domain=[0, 0.4], tick0 = 1, dtick = 1, row=1, col=1)
    fig.update_xaxes(title_text="X axis", domain=[0.6, 1], tick0 = 1, dtick = 1, row=1, col=2)
    fig.update_xaxes(title_text="X axis", domain=[0, 0.4], tick0 = 1, dtick = 1, row=2, col=1)
    fig.update_xaxes(title_text="X axis", domain=[0.6, 1], tick0 = 1, dtick = 1, row=2, col=2)
    fig.update_xaxes(title_text="X axis", domain=[0, 0.4], tick0 = 1, dtick = 1, row=3, col=1)
    fig.update_xaxes(title_text="X axis", domain=[0.6, 1], tick0 = 1, dtick = 1, row=3, col=2)

    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=1, col=1)
    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=1, col=2)
    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=2, col=1)
    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=2, col=2)    
    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=3, col=1)
    fig.update_yaxes(title_text="Y axis", tick0 = 1, dtick = 1, row=3, col=2)
    
    fig.update_layout(height=1350, width = 900, showlegend=False,)

    iplot(fig)

In [17]:
#button 3

button3 = widgets.Button(description="Download", layout={'width': '145px'})
output1 = widgets.Output(layout={'border': '1px solid green', 
                                 'min_width': '920px', 'min_height': '500px',})
output2 = widgets.Output(layout={'border': '1px solid green', 
                                 'min_width': '160px', 'min_height': '40px',})
def on_button_clicked_3(a):
    with output2:
        clear_output()
        csvlist = os.listdir('bin/rawData')
        csvlist.sort()
        filename_Sample ='bin/rawData/'+csvlist[int(Sample_number.value)-1]
        Sample=Download(filename_Sample, style='info')
        display(Sample)

Top = widgets.VBox([input3,button3], layout={'min_width': '165px', 'position':'middle'})
Top_box = widgets.VBox([Top,output2], layout= {'justify_content': 'flex-start'})

#button 1
button1 = widgets.Button(description="Analyze", layout={'width': '145px'})
output1 = widgets.Output(layout={'border': '1px solid green', 
                                 'min_width': '920px', 'min_height': '500px',})


def on_button_clicked_1(a):
     with output1:
        clear_output()
        measurement_analystic(output1)
        filename ='download/'+f.list()[0][11:]
        save=Download(filename, style='info')
        display(save)

#button 2
button2 = widgets.Button(description="Visualize", layout={'width': '145px'})

def on_button_clicked_2(a):
     with output1:
        clear_output()
        new_visualization(output1)
        


out_text = widgets.HTML('<p style="background:lightgray" align="center"> OUTPUTS</p>')
out_section2 = widgets.VBox([out_text, outlabels2])

#display part:
display(f)


#display(button1, output1)
button1.on_click(on_button_clicked_1)


#display(button2, output2)
button2.on_click(on_button_clicked_2)

#display(button2, output2)
button3.on_click(on_button_clicked_3)



left = widgets.VBox([input2,button1], layout={'min_width': '140px', 'position':'middle'})
middle = widgets.VBox([input1,button2], layout={'min_width': '110px', 'position':'middle'})
upper_box = widgets.HBox([Top_box,left, middle,out_section2], layout= {'justify_content': 'flex-start'})
widgets.VBox([upper_box,output1], layout= {'justify_content': 'flex-start'})